In [1]:
import numpy as np
import random
import time
import pandas as pd
from copy import copy

Actions = ['Up', 'Down', 'Left', 'Right']

In [2]:
def build_env_matrix(n):
    env_matrix = np.zeros(shape=(int(n),int(n)))
    T = random.sample(xrange(0,(int(n)-1)), (int(n)-1))
    env_matrix[T[0],T[1]] = 100
    env_matrix[T[2],T[3]] = -100
    x,y = np.where(env_matrix == 0)
    s = [random.choice(x),random.choice(y)]
    env_matrix[s[0],s[1]] = 97
    return env_matrix,s,q_table(n)

In [3]:
def q_table(n):
    q_table = []
    for i in range(0,n):
        q_table.append([])
        for j in range(0,n):
            q_table[i].append([.0,.0,.0,.0])
    return np.array(q_table)

In [4]:
Environment, Start_Mem, Q = build_env_matrix(12)
LEARNING_RATE = 0.5
GLOBAL_REWARD = -0.04
DISCOUNT_FACTOR = 0.99
EPSILON = 0.8

In [5]:
def choose_action(state, q_table, n):
    condition = all(i>=0 and i<int(n) for i in Start)
    if not condition:
        return
    if (np.random.uniform() > EPSILON):
        chosen_action = np.random.choice(Actions)
    else:
        index = np.argmax(q_table[state[0]][state[1]])
        chosen_action = Actions[int(index)]
    return chosen_action

In [6]:
def update_environment(Env, Start, chosen_action, n):
    if chosen_action == 'Up': 
        condition = (((Start[0]-1)>=0) and ((Start[0]-1)<n))
        if condition: 
            Start[0] = (Start[0]-1) 
            if not (int(Env[Start[0]][Start[1]]) == int(100) or int(Env[Start[0]][Start[1]]) == int(-100)):
                if not (int(Env[(Start[0]+1)][Start[1]]) == int(100) or int(Env[(Start[0]+1)][Start[1]]) == int(-100)):
                    Env[(Start[0]+1)][Start[1]] = 0
                    Env[Start[0]][Start[1]] = 97
                else:
                    Env[Start[0]][Start[1]] = 97
            else:
                Env[(Start[0]+1)][Start[1]] = 0 
    elif chosen_action == 'Down': 
        condition = (((Start[0]+1)>=0) and ((Start[0]+1)<n)) 
        if condition: 
            Start[0] = (Start[0]+1) 
            if not (int(Env[(Start[0])][Start[1]]) == int(100) or int(Env[(Start[0])][Start[1]]) == int(-100)):
                if not (int(Env[(Start[0]-1)][Start[1]]) == int(100) or int(Env[(Start[0]-1)][Start[1]]) == int(-100)):
                    Env[(Start[0]-1)][Start[1]] = 0 
                    Env[Start[0]][Start[1]] = 97
                else:
                    Env[Start[0]][Start[1]] = 97
            else:
                Env[Start[0]-1][Start[1]] = 0 
    elif chosen_action == 'Left': 
        condition = (((Start[1]-1)>=0) and ((Start[1]-1)<n))
        if condition:  
            Start[1] = (Start[1]-1) 
            if not (int(Env[Start[0]][Start[1]]) == int(100) or int(Env[Start[0]][Start[1]]) == int(-100)):
                if not (int(Env[Start[0]][(Start[1]+1)]) == int(100) or int(Env[Start[0]][(Start[1]+1)]) == int(-100)):
                    Env[Start[0]][(Start[1]+1)] = 0
                    Env[Start[0]][Start[1]] = 97
                else:
                    Env[Start[0]][Start[1]] = 97
            else:
                Env[Start[0]][(Start[1]+1)] = 0 
    elif chosen_action == 'Right': 
        condition = (((Start[1]+1)>=0) and ((Start[1]+1)<n)) 
        if condition: 
            Start[1] = (Start[1]+1) 
            if not (int(Env[Start[0]][Start[1]]) == int(100) or int(Env[Start[0]][Start[1]]) == int(-100)):
                if not (int(Env[Start[0]][(Start[1]-1)]) == int(100) or int(Env[Start[0]][(Start[1]-1)]) == int(-100)):
                    Env[Start[0]][(Start[1]-1)] = 0 
                    Env[Start[0]][Start[1]] = 97
                else:
                    Env[Start[0]][Start[1]] = 97
            else:
                Env[Start[0]][(Start[1]-1)] = 0 
    return Env, Start

In [7]:
def Reward_func(State, q_table, Action, Env, Terminate, n):
#     print (((Start[0]-1)>=0) and ((Start[0]-1)<6))
    if int(Env[State[0]][State[1]]) == int(100):
        q_table[State[0]][State[1]][0] += LEARNING_RATE * (1-q_table[State[0]][State[1]][0])
        Terminate = True
    elif int(Env[State[0]][State[1]]) == int(-100):
        q_table[State[0]][State[1]][0] += LEARNING_RATE * (-1-q_table[State[0]][State[1]][0])  
        Terminate = True
    else:
        if Action == 'Up': 
            Action_ = [(Start[0]-1), Start[1]]
            condition = (((Start[0]-1)>=0) and ((Start[0]-1)<n)) 
        elif Action == 'Down': 
            Action_ = [(Start[0]+1), Start[1]]
            condition = (((Start[0]+1)>=0) and ((Start[0]+1)<n))
        elif Action == 'Left': 
            Action_ = [Start[0], (Start[1]-1)]
            condition = (((Start[1]-1)>=0) and ((Start[1]-1)<n)) 
        elif Action == 'Right': 
            Action_ = [Start[0], (Start[1]+1)]
            condition = (((Start[1]+1)>=0) and ((Start[1]+1)<n)) 
        if condition:
            q_table[Start[0]][Start[1]][Actions.index(Action)] += LEARNING_RATE * (GLOBAL_REWARD + (EPSILON* np.amax(q_table[Action_[0]][Action_[1]])) - q_table[Start[0]][Start[1]][Actions.index(Action)])
        else:
            q_table[Start[0]][Start[1]][Actions.index(Action)] += LEARNING_RATE * (GLOBAL_REWARD)
    return Terminate                                                                      
                                                                                   
                                                                                   
                                                                                   

In [8]:
NUMBER_OF_ITER = 5000

In [9]:
Env = copy(Environment)
Start = copy(Start_Mem)
print Environment, Start_Mem, len(Environment)

[[   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.  100.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.   97.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [-100.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]] [6, 1] 12


In [18]:
for i in range(0,NUMBER_OF_ITER):
    Env = copy(Environment)
    Start = copy(Start_Mem)
    Terminate = False
    Step = 0
    while not Terminate:
        Action = choose_action(Start, Q, len(Env))
        Terminate = Reward_func(Start, Q, Action, Env, Terminate, len(Env))
        if not Terminate:
            update_environment(Env, Start, Action, len(Env))
            Step += 1

In [19]:
Q

array([[[-0.08      , -0.091951  , -0.08      , -0.083231  ],
        [-0.1       , -0.0964575 , -0.081058  , -0.07827   ],
        [-0.08      , -0.07911   , -0.0873564 , -0.07871   ],
        [-0.08      , -0.084718  , -0.077268  , -0.07455   ],
        [-0.08      , -0.06718   , -0.080292  , -0.06635   ],
        [-0.06      , -0.0526    , -0.063     , -0.0535    ],
        [-0.06      , -0.038     , -0.042     ,  0.0098    ],
        [-0.02      ,  0.146     , -0.028     ,  0.112     ],
        [-0.02      ,  0.5325    ,  0.        ,  0.        ],
        [-0.02      , -0.02      , -0.02      ,  0.        ],
        [-0.02      ,  0.0318    , -0.02      ,  0.        ],
        [-0.04      , -0.02      , -0.02      ,  0.        ]],

       [[-0.0922175 , -0.094069  , -0.14      , -0.0998923 ],
        [-0.090683  , -0.10056574, -0.0947118 , -0.09768755],
        [-0.084835  , -0.084882  , -0.09638052, -0.0871535 ],
        [-0.07535   , -0.090751  , -0.07482   , -0.0891935 ],
      

In [21]:
import time
EPSILON = 0.99
Terminate = False
Env = copy(Environment)
Start = copy(Start_Mem)
Terminate = False
Step = 0
while not Terminate:
    Action = choose_action(Start, Q, len(Env))
    Terminate = Reward_func(Start, Q, Action, Env, Terminate, len(Env))
    if not Terminate:
        update_environment(Env, Start, Action, len(Env))
        Step += 1
    print Env, Action, Step
    time.sleep(0.5)

[[   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.  100.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [-100.   97.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]] Down 1
[[   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    